<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from argo.workflows.sdk import Workflow
from argo.workflows.sdk import task
from argo.workflows.sdk import template

from argo.workflows.sdk.templates import V1Container
from argo.workflows.sdk.templates import V1alpha1Template

In [3]:
import yaml

from pprint import pprint

from argo.workflows.sdk._utils import sanitize_for_serialization

---

In [4]:
!sh -c '[ -f "hello-world-single-task.yaml" ] || curl -LO https://raw.githubusercontent.com/CermakM/argo-python-sdk/master/examples/hello-world-single-task.yaml'

In [5]:
from pathlib import Path

manifest = Path("./hello-world-single-task.yaml").read_text()
print(manifest)

# @file: hello-world-single-task.yaml
apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  name: hello-world
  generateName: hello-world-
spec:
  entrypoint: main
  templates:
  - name: main
    dag:
      tasks:
      - name: A
        template: whalesay

  # @task: [A]
  - name: whalesay
    container:
      name: whalesay
      image: docker/whalesay:latest
      command: [cowsay]
      args: ["hello world"]
status: {}



In [6]:
class HelloWorld(Workflow):
    
    @task
    def A(self) -> V1alpha1Template:
        return self.whalesay()
    
    @template
    def whalesay(self) -> V1Container:
        container = V1Container(
            image="docker/whalesay:latest",
            name="whalesay",
            command=["cowsay"],
            args=["hello world"]
        )
        
        return container

wf = HelloWorld()
wf

{'api_version': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generate_name': 'hello-world-', 'name': 'hello-world'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'A',
                                            'template': 'whalesay'}]},
                         'name': 'main'},
                        {'container': {'args': ['hello world'],
                                       'command': ['cowsay'],
                                       'image': 'docker/whalesay:latest',
                                       'name': 'whalesay'},
                         'name': 'whalesay'}]}}

In [7]:
print(wf.to_yaml())

api_version: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generate_name: hello-world-
  name: hello-world
spec:
  entrypoint: main
  templates:
  - dag:
      tasks:
      - name: A
        template: whalesay
    name: main
  - container:
      args:
      - hello world
      command:
      - cowsay
      image: docker/whalesay:latest
      name: whalesay
    name: whalesay



---

In [8]:
pprint(sanitize_for_serialization(wf))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'hello-world-', 'name': 'hello-world'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'A',
                                            'template': 'whalesay'}]},
                         'name': 'main'},
                        {'container': {'args': ['hello world'],
                                       'command': ['cowsay'],
                                       'image': 'docker/whalesay:latest',
                                       'name': 'whalesay'},
                         'name': 'whalesay'}]},
 'status': {}}


In [9]:
pprint(yaml.safe_load(manifest))

{'apiVersion': 'argoproj.io/v1alpha1',
 'kind': 'Workflow',
 'metadata': {'generateName': 'hello-world-', 'name': 'hello-world'},
 'spec': {'entrypoint': 'main',
          'templates': [{'dag': {'tasks': [{'name': 'A',
                                            'template': 'whalesay'}]},
                         'name': 'main'},
                        {'container': {'args': ['hello world'],
                                       'command': ['cowsay'],
                                       'image': 'docker/whalesay:latest',
                                       'name': 'whalesay'},
                         'name': 'whalesay'}]},
 'status': {}}


In [12]:
assert sanitize_for_serialization(wf) == yaml.safe_load(manifest), "Manifests don't match."